In [60]:
#---Step 0: Import necessary libraries and render plots inline---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.style.use('fivethirtyeight')

In [61]:
user_usage = pd.read_csv("/Users/JCheong/Merge_Join_Data/user_usage.csv")
user_device = pd.read_csv("/Users/JCheong/Merge_Join_Data/user_device.csv")
devices = pd.read_csv("/Users/JCheong/Merge_Join_Data/android_devices.csv")

In [62]:
user_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


In [63]:
user_device.head()

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


In [64]:
android_devices.head()

,Retail Branding,Marketing Name,Device,Model
0,NaN,NaN,AD681H,Smartfren Andromax AD681H
1,NaN,NaN,FJL21,FJL21
2,NaN,NaN,T31,Panasonic T31
3,NaN,NaN,hws7721g,MediaPad 7 Youth 2
4,3Q,OC1020A,OC1020A,OC1020A


There are linking attributes between the sample datasets that are important to note – “use_id” is shared between the user_usage and user_device, and the “device” column of user_device and “Model” column of the devices dataset contain common codes.

# Sample Problem

We would like to determine if the usage patterns for users differ between different devices. For example, do users using Samsung devices use more call minutes than those using  LG devices?

We want to form a single dataframe with columns for user usage figures (calls per month, sms per month etc) and also columns with device information (model, manufacturer, etc).

We will need to “merge” (or “join”) our sample datasets together into one single dataset for analysis.

# Merging Dataframes

“Merging” two datasets is the process of bringing two datasets together into one, and aligning the rows from each based on common attributes or columns.

1. For each row in the user_usage dataset – make a new column that contains the “device” code from the user_devices dataframe. i.e. for the first row, the use_id is 22787, so we go to the user_devices dataset, find the use_id 22787, and copy the value from the “device” column across.

2. After this is complete, we take the new device columns, and we find the corresponding “Retail Branding” and “Model” from the devices dataset.

3. Finally, we can look at different statistics for usage splitting and grouping data by the device manufacturers used.

In [65]:
# Use the default inner merge to combine user_usage and user_device based on use_id
first_merge = pd.merge(user_usage, user_device[["use_id", "platform", "device"]], on="use_id")

first_merge.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


There are three different types of merges available in Pandas. These merge types are common across most database and data-orientated languages (SQL, R, SAS) and are typically referred to as “joins”.

1. Inner Merge / Inner join – The default Pandas behaviour, only keep rows where the merge “on” value exists in both the left and right dataframes.

2. Left Merge / Left outer join – (aka left merge or left join) Keep every row in the left dataframe. Where there are missing values of the “on” variable in the right dataframe, add empty / NaN values in the result.

3. Right Merge / Right outer join – (aka right merge or right join) Keep every row in the right dataframe. Where there are missing values of the “on” variable in the left column, add empty / NaN values in the result.

4. Outer Merge / Full outer join – A full outer join returns all the rows from the left dataframe, all the rows from the right dataframe, and matches up rows where possible, with NaNs elsewhere.

The merge type to use is specified using the “how” parameter in the merge command, taking values “left”, “right”, “inner” (default), or “outer”.

# Example of Left Merge / Left Join

In [66]:
left_merge_test = pd.merge(user_usage,
                           user_device[["use_id", "platform", "device"]],
                           on="use_id",
                           how="left"
                          )

print("There are {} missing values in the result.".format(left_merge_test["device"].isnull().sum()))

There are 81 missing values in the result.


In [67]:
left_merge_test

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F
5,71.59,79.26,1557.33,22793,android,SM-G361F
6,71.59,79.26,519.12,22794,android,SM-G361F
7,71.59,79.26,519.12,22795,android,SM-G361F
8,30.92,22.77,3114.67,22799,android,ONEPLUS A3003
9,69.80,14.70,25955.55,22801,android,GT-I9505


# Outer Merge / Full Outer Join

Finally, we will perform an outer merge using Pandas, also referred to as a “full outer join” or just “outer join”. An outer join can be seen as a combination of left and right joins, or the opposite of an inner join. In outer joins, every row from the left and right dataframes is retained in the result, with NaNs where there are no matched join variables.

As such, we would expect the results to have the same number of rows as there are distinct values of “use_id” between user_device and user_usage, i.e. every join value from the left dataframe will be in the result along with every value from the right dataframe, and they’ll be linked where possible.

In [68]:
unique_use_ids = pd.concat([user_usage["use_id"], user_device["use_id"]], axis=0).unique()

print("There are {} unique values of use_id in our dataframe.".format(unique_use_ids.shape[0]))

outer_merge_test = pd.merge(user_usage,
                            user_device[["use_id", "platform", "device"]],
                            on="use_id",
                            how="outer"
                           )

outer_merge_test

There are 353 unique values of use_id in our dataframe.


,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F
5,71.59,79.26,1557.33,22793,android,SM-G361F
6,71.59,79.26,519.12,22794,android,SM-G361F
7,71.59,79.26,519.12,22795,android,SM-G361F
8,30.92,22.77,3114.67,22799,android,ONEPLUS A3003
9,69.80,14.70,25955.55,22801,android,GT-I9505


In [69]:
NaN_counts = pd.DataFrame(outer_merge_test.apply(lambda x: x.isnull().sum(), axis=1))
counter = (NaN_counts[0] == 0).sum()

print("There are {} rows with no missing values.".format(counter))

There are 159 rows with no missing values.


# Using Indicator Parameter to Track Merges

To assist with the identification of where rows originate from, Pandas provides an “indicator” parameter that can be used with the merge function which creates an additional column called “_merge” in the output that labels the original source for each row.

In [70]:
indicator_test = pd.merge(user_usage,
                          user_device[["use_id", "platform", "device"]],
                          on="use_id",
                          how="outer",
                          indicator=True
                         )

indicator_test

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device,_merge
0,21.97,4.82,1557.33,22787,android,GT-I9505,both
1,1710.08,136.88,7267.55,22788,android,SM-G930F,both
2,1710.08,136.88,7267.55,22789,android,SM-G930F,both
3,94.46,35.17,519.12,22790,android,D2303,both
4,71.59,79.26,1557.33,22792,android,SM-G361F,both
5,71.59,79.26,1557.33,22793,android,SM-G361F,both
6,71.59,79.26,519.12,22794,android,SM-G361F,both
7,71.59,79.26,519.12,22795,android,SM-G361F,both
8,30.92,22.77,3114.67,22799,android,ONEPLUS A3003,both
9,69.80,14.70,25955.55,22801,android,GT-I9505,both


# Final Merge - Joining Device Details to Result

Coming back to our original problem, we have already merged user_usage with user_device, so we have the platform and device for each user. Originally, we used an “inner merge” as the default in Pandas, and as such, we only have entries for users where there is also device information. We’ll redo this merge using a left join to keep all users, and then use a second left merge to finally to get the device manufacturers in the same dataframe.

In [71]:
# First, add the platform and device to the user usage using a left join
result = pd.merge(user_usage,
                  user_device[["use_id", "platform", "device"]],
                  on="use_id",
                  how="left"
                 )

# Second, add manufacturer and model columns based on "device column" from the previous dataframe and the "model" column in the device dataframe
devices.rename(columns={"Retail Branding" : "manufacturer"}, inplace=True)
result = pd.merge(result,
                  devices[["manufacturer","Model"]],
                  left_on="device",
                  right_on="Model",
                  how="left"
                 )

result.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device,manufacturer,Model
0,21.97,4.82,1557.33,22787,android,GT-I9505,Samsung,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F,Samsung,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F,Samsung,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303,Sony,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F,Samsung,SM-G361F


The columns used in a merge operator do not need to be named the same in both the left and right dataframe. In the second merge above, note that the device ID is called “device” in the left dataframe, and called “Model” in the right dataframe.

Different column names are specified for merges in Pandas using the “left_on” and “right_on” parameters, instead of using only the “on” parameter.

# Calculating Statistics Based on Device

With our merges complete, we can use the data aggregation functionality of Pandas to quickly work out the mean usage for users based on device manufacturer. Note that the small sample size creates even smaller groups, so I wouldn’t attribute any statistical significance to these particular results.

In [79]:
aggregation = {"outgoing_mins_per_month" : "mean",
               "outgoing_sms_per_month" : "mean",
               "monthly_mb" : "mean",
               "use_id" : "count"
              }

result.groupby("manufacturer").agg(aggregation)

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
manufacturer,,,,
HTC,299.842955,93.059318,5144.077955,44
Huawei,81.526667,9.500000,1561.226667,3
LGE,111.530000,12.760000,1557.330000,2
Lava,60.650000,261.900000,12458.670000,2
Lenovo,215.920000,12.930000,1557.330000,2
Motorola,95.127500,65.666250,3946.500000,16
OnePlus,354.855000,48.330000,6575.410000,6
Samsung,191.010093,92.390463,4017.318889,108
Sony,177.315625,40.176250,3212.000625,16
